In [1]:
import pandas as pd
import numpy as np
import os
import zipfile
import sqlite3

In [2]:
zip_file = "tests_male_csv2.zip" 

#### Extract Data

In [3]:
df_list = []

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    # Only include CSV files that contain '_info' in the filename
    csv_files = [f for f in zip_ref.namelist() if f.endswith('.csv') and '_info' in f]
    
    for file in csv_files:
        with zip_ref.open(file) as f:
            df_temp = pd.read_csv(f, header=1, usecols=[0, 1, 2])
            df_temp['File Name'] = file
            df_list.append(df_temp)

df_info = pd.concat(df_list, ignore_index=True)

In [4]:
filter_df = df_info[(df_info['balls_per_over'] == 'team') | (df_info['balls_per_over'] == 'season') | (df_info['balls_per_over'] == 'venue') | (df_info['balls_per_over'] == 'toss_winner') | (df_info['balls_per_over'] == 'toss_decision') | (df_info['balls_per_over'] == 'winner') | (df_info['balls_per_over'] == 'outcome')]

In [5]:
filter_df['match_id'] = filter_df['File Name'].apply(lambda x: x.split("_")[0])

C:\Users\User\AppData\Local\Temp\ipykernel_1280\3305250314.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_df['match_id'] = filter_df['File Name'].apply(lambda x: x.split("_")[0])


In [6]:
teams_df = filter_df[filter_df['balls_per_over'] == 'team']

In [7]:
teams_df['team_number'] = teams_df.groupby('match_id').cumcount() + 1

C:\Users\User\AppData\Local\Temp\ipykernel_1280\941047205.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  teams_df['team_number'] = teams_df.groupby('match_id').cumcount() + 1


In [8]:
teams_pivot = teams_df.pivot(index='match_id', columns='team_number', values='6').reset_index()

In [9]:
teams_pivot.columns = ['match_id', 'team1', 'team2'] 

In [10]:
final_df = filter_df.pivot_table(index='match_id',columns='balls_per_over',values='6',aggfunc='first').reset_index()

In [11]:
final_df = final_df.merge(teams_pivot, on='match_id')

In [12]:
final_df.drop(columns=['team'], inplace=True)

In [13]:
final_df = final_df.fillna({'outcome': 'win'})

In [14]:
def remove_after_comma(text):
    if isinstance(text, str):
        return text.split(',')[0]
    return text

In [15]:
final_df['venue'] = final_df['venue'].apply(remove_after_comma)

In [16]:
stadium_map = {
    "W.A.C.A. Ground": "WACA Ground","Western Australia Cricket Association Ground": "WACA Ground",
    "Darren Sammy National Cricket Stadium": "Daren Sammy National Cricket Stadium","M.Chinnaswamy Stadium": "M Chinnaswamy Stadium",
    "Zohur Ahmed Chowdhury Stadium": "Zahur Ahmed Chowdhury Stadium","R.Premadasa Stadium": "R Premadasa Stadium",
    "P Saravanamuttu Stadium": "P Sara Oval","Vidarbha C.A. Ground": "Vidarbha Cricket Association Stadium",
    "New Wanderers Stadium": "The Wanderers Stadium","Feroz Shah Kotla": "Arun Jaitley Stadium",
    "Punjab Cricket Association Stadium": "Punjab Cricket Association IS Bindra Stadium",
}

In [17]:
final_df['venue'] = final_df['venue'].replace(stadium_map)

In [18]:
def remove_after_slash(text):
    if isinstance(text, str):
        return text.split('/')[0]
    return text

In [19]:
final_df['season'] = final_df['season'].apply(remove_after_slash)

#### Export Data

In [21]:
conn = sqlite3.connect('data.db')
final_df.to_sql('train_data', conn, if_exists='replace', index=False)
conn.close()